In [14]:
import sys
import json
import onnxruntime as ort
import pandas as pd
import numpy as np
sys.path.append('../')
from utils.datasets import Datasets
import math
import pandas as pd
import torch.utils.data
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np
import matplotlib.pyplot as plt
import json
from torch.distributions import Categorical
from network.PolicyNetwork import PolicyNetwork
from network.ValueNetwork import ValueNetwork
from utils.datasets import Datasets
import os
import time
from datetime import datetime, timedelta

class Datasets:
    def __init__(self, file_path: str, rank: int = 0, world_size: int = 1) -> None:
        self.file_path = file_path

        # TODO 通过sku信息加载部分结果, 不要全量加载到内存
        print(f"Process {rank}/{world_size}: #############load dataset#############")
        if file_path.endswith(".csv"):
            input_data = pd.read_csv(file_path)
        else:
            input_data = pd.read_parquet(file_path)
        if type(input_data["predicted_demand"].tolist()[0]) == str:
            input_data["predicted_demand"] = input_data["predicted_demand"].apply(eval)
        # input_data = input_data[input_data["idx"] == "257082622036-4894"]

        input_data = input_data.sort_values(by=["idx", "date"])
        print(f"Process {rank}/{world_size}: #############data loaded#############")

        unique_idx = input_data["idx"].unique()
        if world_size > 1:
            # 计算该进程应处理的idx范围
            chunk_size = len(unique_idx) // world_size
            start_idx = rank * chunk_size
            end_idx = start_idx + chunk_size if rank < world_size - 1 else len(unique_idx)
            process_idx_list = unique_idx[start_idx:end_idx]
            input_data = input_data[input_data["idx"].isin(process_idx_list)]
            print(f"Process {rank}/{world_size}: handle {len(process_idx_list)} SKU (Total: {len(unique_idx)})")
        print(f"Process {rank}/{world_size}: #############gen data#############")

        idx_data = input_data.groupby("idx")
        self.sales_map = idx_data["actual"].apply(list).to_dict()
        self.total_sales = input_data["actual"].sum()

        self.predicts_map = idx_data["predicted_demand"].apply(lambda s: np.array(s.tolist())).to_dict()

        self.end_date_map = idx_data["date"].count().to_dict()
        self.leadtime_map = idx_data["leadtime"].apply(list).to_dict()
        self.predict_leadtime_day = idx_data["pred_y"].apply(list).to_dict()
        self.initial_stock_map = idx_data["initial_stock"].max().to_dict()

        date_idx_data = input_data.groupby(["date", "idx"])
        self.sku_id_ls = date_idx_data["pred_y"].sum().unstack().columns.tolist()
        self.predicted_demand = date_idx_data["pred_y"].sum().unstack().fillna(0).values
        print(f"Process {rank}/{world_size}: #############gen data done#############")

    def get_initial_stock_map(self):
        return self.initial_stock_map

    def get_end_date_map(self):
        return self.end_date_map

    def sku_ids(self) -> list:
        return self.sku_id_ls

    def sku_lead_time(self, sku_id: int, day_idx: int) -> int:
        return self.leadtime_map.get(sku_id, [1])[day_idx]

    def range_lead_time(self, day_st, day_ed, sku_id) -> list:
        return self.leadtime_map[sku_id][day_st:day_ed]

    def range_prdicts(self, day_st, day_ed, sku_id) -> list:
        return self.predicts_map[sku_id][day_st:day_ed]

    def get_predicts(self, day_idx: int, sku_id: int) -> list:
        return self.predicts_map[sku_id][day_idx].tolist()

    def get_leadtime_predict(self, day_idx: int, sku_id: int) -> list:
        return self.predict_leadtime_day[sku_id][day_idx]

    def range_sales(self, day_st, day_ed, sku_id) -> list:
        return self.sales_map[sku_id][day_st:day_ed]

    def get_sales(self, day_idx: int, sku_id: int):
        return self.sales_map[sku_id][day_idx]


class ReplenishAgent_make_date:
    def __init__(self, replenish_model, config):
        self.replenish_model = replenish_model
        self.task_name = config["task_name"]
        self.device = config["device"]
        self.state_label = config["state_label"]
        # 分布式训练参数
        self.distributed = False
        self.rank = 0
        self.world_size = 1
        self.is_master = self.rank == 0

        # 检查device是否为数字（GPU索引）并处理
        if isinstance(self.device, int) and not torch.cuda.is_available():
            print(f"Warning: CUDA not available, switching to CPU")
            self.device = "cpu"

        ###输入的数据
        ###商品清单
        self.datasets = Datasets(config["data_path"], self.rank, self.world_size)
        self.sku_id_ls = self.datasets.sku_ids()
        self.predict_leadtime_day = self.datasets.predict_leadtime_day
        self.leadtime_map = self.datasets.leadtime_map
        self.initial_stock_map = self.datasets.get_initial_stock_map()
        self.end_date_map = self.datasets.get_end_date_map()
        self.total_sales = self.datasets.total_sales
        self.sales_map = self.datasets.sales_map

        ###network 配置
        self.state_dim = config["state_dim"]
        self.multiplier_ls: list[float] = config["multiplier_ls"]
        self.action_dim = len(config["multiplier_ls"])
        self.policy = PolicyNetwork(self.state_dim, self.action_dim).to(self.device)
        self.value = ValueNetwork(self.state_dim).to(self.device)

        # # 如果使用分布式训练，将模型包装为DDP模型
        # if self.distributed:
        #     torch.manual_seed(42 + self.rank)
        #     # 检查是否可以使用CUDA
        #     if torch.cuda.is_available() and self.device != "cpu":
        #         self.policy = DDP(self.policy, device_ids=[self.device], output_device=self.device)
        #         self.value = DDP(self.value, device_ids=[self.device], output_device=self.device)
        #     else:
        #         # CPU上使用DDP
        #         self.policy = DDP(self.policy)
        #         self.value = DDP(self.value)

        # # 仅在主进程中创建目录和处理文件
        # if not self.distributed or self.is_master:
        #     os.makedirs(os.path.join("output", self.task_name), exist_ok=True)

        # self.policy_model_filename = os.path.join("output", self.task_name, f"repl_policy_model.pth")
        # self.value_model_filename = os.path.join("output", self.task_name, f"repl_value_model.pth")

        # ###模型训练参数
        # adjusted_lr = config["lr"] * math.sqrt(self.world_size) if self.distributed else config["lr"]

        # self.optimizer = optim.Adam(self.policy.parameters(), lr=adjusted_lr)
        # self.value_optimizer = optim.Adam(self.value.parameters(), lr=adjusted_lr)
        # self.gamma = config["gamma"]
        # self.k_epochs = config["k_epochs"]  ###k_epochs
        # self.eps_clip = config["eps_clip"]  ###eps_clip
        # self.batch_size = config["batch_size"]  ###batch_size
        # self.max_episodes = config["max_episodes"]

        # 存储训练数据
        ###v {sku:[] for sku in self.sku_id_ls}
        self.states_map = {sku: [] for sku in self.sku_id_ls}
        self.actions_map = {sku: [] for sku in self.sku_id_ls}
        # self.logprobs_map = {sku: [] for sku in self.sku_id_ls}
        # self.state_values_map = {sku: [] for sku in self.sku_id_ls}
        # self.rewards_map = {sku: [] for sku in self.sku_id_ls}
        # self.dones_map = {sku: [] for sku in self.sku_id_ls}
        # self.episode_rewards_map = {sku: [] for sku in self.sku_id_ls}
        # self.episode_rewards = []

        # if not self.distributed or self.is_master:
        #     print("initialize done")

    def verify_ddp_gradients(self, model, rank):
        """打印并验证各层梯度"""
        for name, param in model.named_parameters():
            if param.requires_grad and param.grad is not None:
                # 将梯度转换为CPU上的NumPy数组以便打印
                grad_data = param.grad.detach().cpu().numpy()
                grad_sum = np.sum(grad_data)
                grad_mean = np.mean(grad_data)
                print(f"Rank {rank}, Layer {name}: grad_sum={grad_sum:.6f}, grad_mean={grad_mean:.6f}")

    def get_update_action(self, sku_id, state):
        state, action, action_detach, logprob, state_value = self.select_action(state)
        self.states_map[sku_id].append(state)
        self.actions_map[sku_id].append(action_detach)
        self.logprobs_map[sku_id].append(logprob)
        self.state_values_map[sku_id].append(state_value)
        return action

    def select_action(self, state):
        """选择动作"""
        state = torch.tensor(state, dtype=torch.float32).to(self.device)
        probs = self.policy(state)
        m = Categorical(probs)
        action = m.sample()
        logprob = m.log_prob(action).detach()

        # return state, action.item(), action.detach(), logprob, state_value
        value = self.value(state)
        return state, action.item(), action.detach(), logprob, value

    def select_action_deterministic(self, state):
        """确定性地选择动作，每次选择最优的动作"""
        with torch.no_grad():
            state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(self.device)
            probs = self.policy(state)  # 解包返回值，只使用概率
            action = torch.argmax(probs).item()
        return action

    def select_multiplier_deterministic(self, state):
        """调用模型选择最优的multiplier"""
        with torch.no_grad():
            state = torch.tensor(state, dtype=torch.float32).unsqueeze(0).to(self.device)
            probs = self.policy(state)  # 解包返回值，只使用概率
            action = torch.argmax(probs).item()
        return self.multiplier_ls[int(action)]

    def plot_rewards(self):
        """绘制奖励趋势图并保存"""
        # 设置中文字体
        # plt.rcParams["font.sans-serif"] = ["Arial Unicode MS"]  # Mac系统
        # plt.rcParams["axes.unicode_minus"] = False  # 解决负号显示问题
        print(f"max episode_rewards = {max(self.episode_rewards)}")
        data = np.array(self.episode_rewards)
        plt.figure(figsize=(12, 6))
        plt.plot(data, linewidth=1)
        # 使用英文标题避免字体问题
        plt.title("Reward Trend for : " + self.task_name)
        plt.xlabel("Episodes")
        plt.ylabel("Rewards")
        plt.grid(True, linestyle="--", alpha=0.7)
        # 生成时间戳文件名并保存图片
        from datetime import datetime

        # timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"output/{self.task_name}/reward_trend.jpg"
        plt.savefig(filename)
        plt.close()

    ###TODO
    def gen_network_state_v1(self, state_dict, sku, day_idx):
        """
        生成输入network的state：[end_of_stock,next_day_arrive_order,next_day_rts,forecast,leadtime]
        :return:
        """
        ##天数
        ###day_idx=state_dict["day_idx"]
        ##new_state=[day_idx,state_dict["end_of_stock"],state_dict["arrival_abo"],state["arrival_qty"],self.predict_leadtime_day[sku][day_idx]]

        transit_stock = state_dict["transit_stock"] + [0] * (5 - len(state_dict["transit_stock"]))
        new_state = [
            state_dict["end_of_stock"],
            ##state_dict["transit_stock"],
            0,##state_dict["next_day_rts"],
            self.predict_leadtime_day[sku][day_idx],
            self.leadtime_map[sku][day_idx],
            transit_stock[0],
            transit_stock[1],
            transit_stock[2],
            transit_stock[3],
            transit_stock[4],
        ]
        return new_state
    def gen_network_state(self, state_dict, sku, day_idx):
        """
        生成输入network的state：[end_of_stock,next_day_arrive_order,next_day_rts,forecast,leadtime]
        :return:
        """
        ##天数
        ###day_idx=state_dict["day_idx"]
        ##new_state=[day_idx,state_dict["end_of_stock"],state_dict["arrival_abo"],state["arrival_qty"],self.predict_leadtime_day[sku][day_idx]]

        transit_stock = state_dict["transit_stock"] + [0] * (5 - len(state_dict["transit_stock"]))
        new_state = [
            state_dict["end_of_stock"],
            ##state_dict["transit_stock"],
            state_dict["next_day_rts"],
            self.predict_leadtime_day[sku][day_idx],
            self.leadtime_map[sku][day_idx],
            transit_stock[0],
            transit_stock[1],
            transit_stock[2],
            transit_stock[3],
            transit_stock[4],
        ]
        return new_state

    # def reset_network_state(self, sku):
    #     """
    #     生成输入network的state
    #     :return:期末库存，预测值
    #     """
    #     ##return [0, 0, self.predict_leadtime_day[sku][0], self.leadtime_map[sku][0]]
    #     return [
    #         self.initial_stock_map[sku],
    #         0,
    #         self.predict_leadtime_day[sku][0],
    #         self.leadtime_map[sku][0],
    #         0,
    #         0,
    #         0,
    #         0,
    #         0,
    #     ]
    def reset_network_state(self, sku):
        """
        生成输入network的state
        :return:期末库存，预测值
        """
        if self.state_label == "with_rts_split":
            return [self.initial_stock_map[sku], 0, 0, 0, 0, 0, 0] + [self.leadtime_map[sku][0],self.predict_leadtime_day[sku][0]]
        elif self.state_label == "with_rts_combine":
            return [self.initial_stock_map[sku], 0, 0, 0, 0, 0] + [self.leadtime_map[sku][0],self.predict_leadtime_day[sku][0]]
        else:
            return [self.initial_stock_map[sku], 0, 0, 0, 0, 0] + [self.leadtime_map[sku][0],self.predict_leadtime_day[sku][0]]
            
    def reset_day_idx(self):
        return 0

    def reset_state_dict(self):
        ###补货时间，期末库存，预测值
        return {"end_of_stock": 0, "arrival_abo": 0, "arrival_qty": 0, "day_idx": 0}

    def gen_update_data(self):

        states_ls = []
        actions_ls = []
        logprobs_ls = []
        advantages_ls = []
        returns_ls = []
        for sku in self.sku_id_ls:
            ##print(len(self.states_map[sku]),self.leadtime_map[sku][0])
            states = torch.stack(self.states_map[sku]).to(self.device)
            actions = torch.tensor(self.actions_map[sku], dtype=torch.int64).to(self.device)
            logprobs = torch.stack(self.logprobs_map[sku]).to(self.device)
            state_values = torch.cat(self.state_values_map[sku]).squeeze().to(self.device)
            rewards = torch.tensor(self.rewards_map[sku], dtype=torch.float32).to(self.device)
            dones = torch.tensor(self.dones_map[sku], dtype=torch.float32).to(self.device)
            returns = []
            discounted_reward = 0
            for reward, done in zip(reversed(rewards), reversed(dones)):
                if done:
                    discounted_reward = 0
                discounted_reward = reward + (self.gamma * discounted_reward)
                returns.insert(0, discounted_reward)
            returns = torch.tensor(returns, dtype=torch.float32).to(self.device)
            advantages = returns - state_values.detach()

            states_ls.append(states)
            actions_ls.append(actions)
            logprobs_ls.append(logprobs)
            returns_ls.append(returns)
            advantages_ls.append(advantages)

        states_all = torch.cat(states_ls)
        actions_all = torch.cat(actions_ls)
        logprobs_all = torch.cat(logprobs_ls)
        returns_all = torch.cat(returns_ls)
        advantages_all = torch.cat(advantages_ls)
        ### # 标准化优势值
        advantages_all = (advantages_all - advantages_all.mean()) / (advantages_all.std() + 1e-8)

        return states_all, actions_all, logprobs_all, returns_all, advantages_all

    def select_action_batch(self, states_batch):
        """批量选择动作"""
        probs, state_values = self.policy(states_batch)
        m = Categorical(probs)
        actions = m.sample()
        logprobs = m.log_prob(actions)
        values = self.value(states_batch)

        return (states_batch, actions, actions.detach(), logprobs.detach(), values)

    def update(self):
        """更新策略网络"""
        ###生成数据
        states, actions, logprobs, returns, advantages = self.gen_update_data()

        ### PPO更新
        # 将所有经验数据合并为一个数据集
        dataset = torch.utils.data.TensorDataset(states, actions, logprobs, advantages, returns)

        # 在分布式训练中使用DistributedSampler
        if self.distributed:
            sampler = DistributedSampler(dataset, num_replicas=self.world_size, rank=self.rank)
            dataloader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, sampler=sampler)
        else:
            dataloader = torch.utils.data.DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        # PPO更新
        for epoch in range(self.k_epochs):
            # 在每个epoch开始前设置sampler的epoch
            if self.distributed:
                dataloader.sampler.set_epoch(epoch)

            for batch in dataloader:
                (
                    batch_states,
                    batch_actions,
                    batch_logprobs,
                    batch_advantages,
                    batch_returns,
                ) = batch

                batch_probs = self.policy(batch_states)  # 获取动作的概率分布

                batch_values = self.value(batch_states)
                m = Categorical(batch_probs)
                batch_new_logprobs = m.log_prob(batch_actions)  ## # 计算采样动作的对数概率
                entropy = m.entropy().mean()  ##计算分布的熵
                # Finding the ratio
                ratios = torch.exp(batch_new_logprobs - batch_logprobs)
                # Finding Surrogate Loss
                surr1 = ratios * batch_advantages
                surr2 = torch.clamp(ratios, 1 - self.eps_clip, 1 + self.eps_clip) * batch_advantages
                # final loss of clipped objective PPO
                loss = -torch.min(surr1, surr2).mean() - 0.01 * entropy
                loss_value = nn.MSELoss()(batch_values.squeeze(), batch_returns)

                ###更新网络
                self.optimizer.zero_grad()
                self.value_optimizer.zero_grad()
                loss.backward()
                loss_value.backward()
                self.optimizer.step()
                self.value_optimizer.step()

            # 每个 epoch 保存一次模型，仅在主进程中进行
            if not self.distributed or self.is_master:
                self.save_model(self.policy_model_filename)
                self.save_value_model(self.value_model_filename)

        # 清空缓存
        self.states_map = {sku: [] for sku in self.sku_id_ls}
        self.actions_map = {sku: [] for sku in self.sku_id_ls}
        self.logprobs_map = {sku: [] for sku in self.sku_id_ls}
        self.state_values_map = {sku: [] for sku in self.sku_id_ls}
        self.rewards_map = {sku: [] for sku in self.sku_id_ls}
        self.dones_map = {sku: [] for sku in self.sku_id_ls}
        self.episode_rewards_map = {sku: [] for sku in self.sku_id_ls}


    def cal_date(self, start_date, delta_days):
        end_date = datetime.strptime(start_date, "%Y-%m-%d") + timedelta(days=delta_days)
        return end_date.strftime("%Y-%m-%d")

    def gen_multipliers(self, actions_map, start_date, task_name):

        key_ls = []
        res_data_ls = []

        for key, value in actions_map.items():
            key_ls.append(key)
            res_data_ls.append(value)

        df = pd.DataFrame(data={"idx": key_ls, "multilpier": res_data_ls})

        df["multiplier_index"] = df["multilpier"].apply(lambda x: list(range(len(x))))

        exploded_df = df.explode(["multilpier", "multiplier_index"])
        exploded_df["ds"] = exploded_df.apply(lambda s: self.cal_date(start_date, s["multiplier_index"]), axis=1)
        exploded_df["dt_version"] = task_name
        exploded_df = exploded_df[["idx", "ds", "multilpier", "dt_version"]]
        exploded_df.columns = ["idx", "ds", "multilpier", "dt_version"]
        return exploded_df


    def save_model(self, path):
        """保存模型和网络配置"""
        # 只在主进程保存模型
        if self.distributed and not self.is_master:
            return

        model_state_dict = self.policy.module.state_dict() if self.distributed else self.policy.state_dict()
        model_info = {
            "state_dict": model_state_dict,
            "network_type": "",
            "state_dim": self.state_dim,
            "action_dim": self.action_dim,
        }
        torch.save(model_info, path)

    def save_value_model(self, path):
        """保存模型和网络配置"""
        # 只在主进程保存模型
        if self.distributed and not self.is_master:
            return

        model_state_dict = self.value.module.state_dict() if self.distributed else self.value.state_dict()
        model_info = {
            "state_dict": model_state_dict,
            "network_type": "",
            "state_dim": self.state_dim,
        }
        torch.save(model_info, path)

    def load_model(self, path):
        """加载模型和网络配置"""
        model_info = torch.load(path, map_location=self.device)
        # 加载模型参数
        if self.distributed:
            self.policy.module.load_state_dict(model_info["state_dict"])
        else:
            self.policy.load_state_dict(model_info["state_dict"])
        self.policy.to(self.device)


In [35]:
# 率先定义onnx模型
onnx_path = "/home/work/apb-project/ais-deploy-demo-cache/replenishment-service-file/rl/id/default_test/model.onnx"

# 读数据
cfg_path = "/home/work/apb-project/ais-deploy-demo-cache/replenishment-service-file/rl/id/default_test/model.json"
with open(cfg_path, "r") as f:
    cfg = json.load(f)
#cfg["data_path"] = "/home/work/apb-project/ais-deploy-demo-cache/replenishment/data/20250319/dwd_cache_replenish_rl_test_dataset_1k_idx_forward_period_g1"
a = pd.read_parquet(cfg["data_path"])
print(a.head(10))
ra = ReplenishAgent_make_date(None, cfg)
#ra.load_model(model_path)
state_map = {sku: ra.reset_network_state(sku) for sku in ra.sku_id_ls}



# onnx模型的预测流程，暂时注释，后续再启动
# 加载onnx模型
sess_options = ort.SessionOptions()
# 设置单个操作内部使用的线程数
sess_options.intra_op_num_threads = 10
# 设置是否顺序执行操作图内部的算子，还是并行执行
sess_options.execution_mode = ort.ExecutionMode.ORT_SEQUENTIAL
onnx_model = ort.InferenceSession(onnx_path, sess_options=sess_options)
onnx_sku_list = []
onnx_action_list = []
onnx_action_proba_list = []
for sku in ra.sku_id_ls[0:2]:
    print(state_map[sku])
    onnx_action = onnx_model.run(None,{'x':np.array(state_map[sku])[np.newaxis,:].astype(np.float32)})[0]
    onnx_action_list.append(np.argmax(onnx_action))
    onnx_action_proba_list.append(onnx_action)
    onnx_sku_list.append(sku)

onnx_sku = np.array(onnx_sku_list)
onnx_actions = np.array(onnx_action_list)
onnx_actions_proba = np.concatenate(onnx_action_proba_list, axis=0)
print(onnx_sku.shape)
print(onnx_actions)
print(onnx_actions_proba)
# 加载数据
datasets = Datasets("/home/work/apb-project/ais-deploy-demo-cache/replenishment/data/20250319/dwd_cache_replenish_rl_test_dataset_1k_idx_forward_period_g1")


               idx  date   repl_date  leadtime  actual  pred_y  \
0  59686521620-565     0  2025-01-16         2       0     1.0   
1  59686521620-565     1  2025-01-17         2       1     1.0   
2  59686521620-565     2  2025-01-18         2       3     1.0   
3  59686521620-565     3  2025-01-19         2       1     1.0   
4  59686521620-565     4  2025-01-20         2       1     1.0   
5  59686521620-565     5  2025-01-21         2       2     1.0   
6  59686521620-565     6  2025-01-22         2       0     1.0   
7  59686521620-565     7  2025-01-23         2       1     2.0   
8  59686521620-565     8  2025-01-24         2       0     1.0   
9  59686521620-565     9  2025-01-25         2       0     1.0   

                 predicted_demand  initial_stock     model_id last_soc  ...  \
0  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]            0.0  59686521620      565  ...   
1  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]            0.0  59686521620      565  ...   
2  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]   

In [36]:
import requests
ModelVersion = "default_test"
columns = cfg["columns"]
tmp_list = []
tmp_dict = {}
tmp_dict["logid"] = 123456
tmp_dict["clientip"] = "winning"
tmp_dict["data"] = {"ModelVersion": ModelVersion}

for sku in ra.sku_id_ls[0:2]:
    feature = dict(zip(columns, [float(i) for i in state_map[sku]]))
    print(feature)
    temp_dict = {}
    temp_dict["info"] = {"unique_id": sku, "outlier_detection":0, "split_2_daily":0}
    temp_dict["features"] = feature
    tmp_list.append(temp_dict)
tmp_dict["data"]["input"] = tmp_list

url = "http://127.0.0.1:1145/api/process"
#url = "http://sg10.aip.mlp.shopee.io/aip-svc-100113/rl-20250324/api/process"
req = json.dumps(tmp_dict)
with open("/home/work/apb-project/rl_req.json",'w') as f:
    f.write(req)
    f.close()
res=requests.post(url = url, data=req)

b = json.dumps(res.json())
with open("/home/work/apb-project/rl_req.json",'w') as f:
    f.write(req)
    f.close()

{'end_of_stock': 0.0, 'transit_stock_0': 0.0, 'transit_stock_1': 0.0, 'transit_stock_2': 0.0, 'transit_stock_3': 0.0, 'transit_stock_4': 0.0, 'leadtime': 2.0, 'pred_y': 1.0}
{'end_of_stock': 0.0, 'transit_stock_0': 0.0, 'transit_stock_1': 0.0, 'transit_stock_2': 0.0, 'transit_stock_3': 0.0, 'transit_stock_4': 0.0, 'leadtime': 2.0, 'pred_y': 3.0}


In [37]:
b

'{"err_num": 0, "err_msg": "OK", "host": "vs-0224df45-190f-4e64-b2fe-80ea263d8380-4182861100-f448f97bjt8q", "result": {"output": [{"unique_id": "10046901900-18299", "output": {"multiplier": 3, "probability": [0.12013761699199677, 0.10723525285720825, 0.10806940495967865, 0.12380103766918182, 0.11115903407335281, 0.14174553751945496, 0.1380762904882431, 0.14977578818798065]}}, {"unique_id": "100578503849-14461", "output": {"multiplier": 2, "probability": [0.12818120419979095, 0.09438678622245789, 0.09851526468992233, 0.12353232502937317, 0.11343301832675934, 0.13322000205516815, 0.17156317830085754, 0.13716810941696167]}}], "ModelVersion": "default_test"}}'

In [14]:
onnx_action_proba_list = [i.squeeze().tolist() for i in onnx_action_proba_list]

AttributeError: 'list' object has no attribute 'squeeze'

In [15]:
predict_list = res.json()['result']['output']
predict_list = [pd.DataFrame(i) for i in predict_list]

In [10]:
predict_df = pd.concat(predict_list).reset_index(names="value")
a = predict_df.query("value=='probability'")
a["offline_output"] = onnx_action_proba_list

/tmp/ipykernel_11580/3736755721.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a["offline_output"] = onnx_action_proba_list


In [11]:
a

,value,unique_id,output,offline_output
1,probability,101724530018-26,"[0.166314959526062, 0.1331610530614853, 0.1388...","[0.166314959526062, 0.1331610530614853, 0.1388..."
3,probability,102460392589-26,"[0.19042392075061798, 0.12624157965183258, 0.1...","[0.19042392075061798, 0.12624157965183258, 0.1..."
5,probability,104185028940-14461,"[0.16609469056129456, 0.13671071827411652, 0.1...","[0.16609469056129456, 0.13671071827411652, 0.1..."
7,probability,104357715697-9820,"[0.1883750855922699, 0.12450452893972397, 0.13...","[0.1883750855922699, 0.12450452893972397, 0.13..."
9,probability,105514554826-2240,"[0.1876506209373474, 0.12993977963924408, 0.15...","[0.1876506209373474, 0.12993977963924408, 0.15..."
...,...,...,...,...
1991,probability,98986994632-14461,"[0.166314959526062, 0.1331610530614853, 0.1388...","[0.166314959526062, 0.1331610530614853, 0.1388..."
1993,probability,98987885103-10631,"[0.1876506209373474, 0.12993977963924408, 0.15...","[0.1876506209373474, 0.12993977963924408, 0.15..."
1995,probability,99588195830-1581,"[0.20344261825084686, 0.12024938315153122, 0.1...","[0.20344261825084686, 0.12024938315153122, 0.1..."
1997,probability,99600246322-2240,"[0.1876506209373474, 0.12993977963924408, 0.15...","[0.1876506209373474, 0.12993977963924408, 0.15..."


In [7]:
predict_df

,value,unique_id,output
0,multiplier,101724530018-26,[1.8]
1,probability,101724530018-26,"[0.166314959526062, 0.1331610530614853, 0.1388..."
2,multiplier,102460392589-26,[1.8]
3,probability,102460392589-26,"[0.19042392075061798, 0.12624157965183258, 0.1..."
4,multiplier,104185028940-14461,[1.8]
...,...,...,...
1995,probability,99588195830-1581,"[0.20344261825084686, 0.12024938315153122, 0.1..."
1996,multiplier,99600246322-2240,[1.8]
1997,probability,99600246322-2240,"[0.1876506209373474, 0.12993977963924408, 0.15..."
1998,multiplier,99670062948-764,[1.8]


In [72]:
a["output"] == a["offline_output"]

In [29]:
json_path = "/home/work/apb-project/rl_req.json"
with open(json_path,'rb') as f:
    inputs = json.load(f)
print(inputs["data"]["input"][0])
print(inputs["data"]["input"][1])
print(inputs["data"])
raw = inputs["data"]["input"][0]

inputs["data"]["input"] = [raw for i in range(16)]
req = json.dumps(inputs)

{'info': {'unique_id': '101724530018-26', 'outlier_detection': 0, 'split_2_daily': 0}, 'features': {'input': [0.0, 0, 1.0, 1, 0, 0, 0, 0, 0]}}
{'info': {'unique_id': '102460392589-26', 'outlier_detection': 0, 'split_2_daily': 0}, 'features': {'input': [0.0, 0, 1.0, 2, 0, 0, 0, 0, 0]}}
{'ModelVersion': '20250324', 'input': [{'info': {'unique_id': '101724530018-26', 'outlier_detection': 0, 'split_2_daily': 0}, 'features': {'input': [0.0, 0, 1.0, 1, 0, 0, 0, 0, 0]}}, {'info': {'unique_id': '102460392589-26', 'outlier_detection': 0, 'split_2_daily': 0}, 'features': {'input': [0.0, 0, 1.0, 2, 0, 0, 0, 0, 0]}}]}


In [30]:
import requests
url = "http://sg10.aip.mlp.shopee.io/aip-svc-100113/rl-20250324/api/process"
res=requests.post(url = url, data=req)
#print(inputs["data"])
print(res.json())

{'ModelVersion': '20250324', 'input': [{'info': {'unique_id': '101724530018-26', 'outlier_detection': 0, 'split_2_daily': 0}, 'features': {'input': [0.0, 0, 1.0, 1, 0, 0, 0, 0, 0]}}, {'info': {'unique_id': '101724530018-26', 'outlier_detection': 0, 'split_2_daily': 0}, 'features': {'input': [0.0, 0, 1.0, 1, 0, 0, 0, 0, 0]}}, {'info': {'unique_id': '101724530018-26', 'outlier_detection': 0, 'split_2_daily': 0}, 'features': {'input': [0.0, 0, 1.0, 1, 0, 0, 0, 0, 0]}}, {'info': {'unique_id': '101724530018-26', 'outlier_detection': 0, 'split_2_daily': 0}, 'features': {'input': [0.0, 0, 1.0, 1, 0, 0, 0, 0, 0]}}, {'info': {'unique_id': '101724530018-26', 'outlier_detection': 0, 'split_2_daily': 0}, 'features': {'input': [0.0, 0, 1.0, 1, 0, 0, 0, 0, 0]}}, {'info': {'unique_id': '101724530018-26', 'outlier_detection': 0, 'split_2_daily': 0}, 'features': {'input': [0.0, 0, 1.0, 1, 0, 0, 0, 0, 0]}}, {'info': {'unique_id': '101724530018-26', 'outlier_detection': 0, 'split_2_daily': 0}, 'features'